In [1]:
import random

import load_data
import numpy as np
import stylized_score

## Set seeds for reconstruction of the results

In [2]:
# Set seeds.
SEED = 12345
np.random.seed(SEED)
random.seed(SEED)

#### Load Data

Filter out selected times and stocks

In [3]:
min_len = 9216
price_df = load_data.load_prices("sp500")
log_ret, symbols = load_data.get_log_returns(price_df, min_len, True)

Timespan 1987-12-17 -- 2024-07-17
Number of symbols 234


In [15]:
B = 500
S = 8
L = 4096
R = 5

scores = []
total_scores = []
for _ in range(R):
    n_sym = len(symbols)
    all_idx = set(range(n_sym))
    idx_one = list(np.random.choice(list(all_idx), n_sym // 2))
    idx_two = list(all_idx.difference(idx_one))

    stf_one = stylized_score.boostrap_stylized_facts(log_ret[:, idx_one], B, S, L)
    stf_two = stylized_score.boostrap_stylized_facts(log_ret[:, idx_two], B, S, L)

    tot, sc, _ = stylized_score.stylized_score(stf_one, stf_two)

    scores.append(sc)
    total_scores.append(tot)

In [29]:
B = 500
S = 8
L = 4096

stock_scores = []
stock_total_scores = []
stf = stylized_score.boostrap_stylized_facts(log_ret, B, S, L)
n_sym = len(symbols)
for i in range(n_sym):
    stf_two = stylized_score.boostrap_stylized_facts(log_ret[:, [i]], B, S, L)
    tot, sc, _ = stylized_score.stylized_score(stf, stf_two)

    stock_scores.append(sc)
    stock_total_scores.append(tot)

In [41]:
B = 500
S = 8
L = 3072

index_data = []

stf_sp = stylized_score.boostrap_stylized_facts(log_ret, B, S, L)

smi = load_data.load_log_returns("smi", 4096)
stf_smi = stylized_score.boostrap_stylized_facts(smi, B, S, L)

dax = load_data.load_log_returns("dax", 4096)
stf_dax = stylized_score.boostrap_stylized_facts(dax, B, S, L)

tot, sc, _ = stylized_score.stylized_score(stf_dax, stf_smi)
index_data.append((tot, sc, "DAX -- SMI"))
tot, sc, _ = stylized_score.stylized_score(stf_dax, stf_sp)
index_data.append((tot, sc, "DAX -- S\\&P500"))
tot, sc, _ = stylized_score.stylized_score(stf_smi, stf_dax)
index_data.append((tot, sc, "SMI -- DAX"))
tot, sc, _ = stylized_score.stylized_score(stf_smi, stf_sp)
index_data.append((tot, sc, "SMI -- S\\&P500"))
tot, sc, _ = stylized_score.stylized_score(stf_sp, stf_dax)
index_data.append((tot, sc, "S\\&P500 -- DAX"))
tot, sc, _ = stylized_score.stylized_score(stf_sp, stf_smi)
index_data.append((tot, sc, "S\\&P500 -- SMI"))

# Print Tables for thesis

In [40]:
table_start = r"""\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{c|l|l|l|l|l|l|l}
            \toprule
            Base -- Comp & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\"""
table_end = """            \\bottomrule
        \\end{tabular}
        \\begin{tablenotes}[flushleft]
            \\footnotesize
            \\item Stylized Scores between market distributions.
             The results are sorted by the base index first and then by the
             compared index.
        \\end{tablenotes}
  \\end{threeparttable}
\\end{table}"""

print(table_start)
for mu, scs, run in index_data:
    ind_scores_str = " & ".join([f"${d:.2f}$" for d in scs])
    print(r"            \hline")
    print(f"            {run} & {mu:.2f} & {ind_scores_str}\\\\")

print(table_end)

\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{c|l|l|l|l|l|l|l}
            \toprule
            Base -- Comp & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\
            \hline
            S\&P500 -- SMI & 1.13 & $2.87$ & $0.29$ & $0.83$ & $1.67$ & $0.68$ & $0.44$\\
            \hline
            S\&P500 -- DAX & 1.17 & $2.62$ & $0.26$ & $1.20$ & $1.93$ & $0.57$ & $0.44$\\
            \hline
            SMI -- DAX & 0.60 & $0.92$ & $0.14$ & $0.29$ & $0.83$ & $0.69$ & $0.72$\\
            \hline
            SMI -- S\&P500 & 0.84 & $1.70$ & $0.38$ & $0.62$ & $1.21$ & $0.67$ & $0.45$\\
            \hline
            DAX -- S\&P500 & 0.80 & $1.61$ & $0.31$ & $0.85$ & $1.19$ & $0.46$ & $0.38$\\
            \hline
            DAX -- SMI & 0.54 & $0.96$ & $0.13$ & $0.28$ & $0.70$ & $0.57$ & $0.62$\\
  

In [34]:
stock_data = zip(stock_total_scores, stock_scores, symbols)
stock_data = sorted(stock_data, key=lambda x: x[0])
avg = (np.mean(stock_total_scores), tuple(np.mean(stock_scores, axis=0)), "AVG")
stock_data = list(reversed(list(reversed(stock_data)) + [avg]))
max = stock_data[-1]
max = (max[0], max[1], "MAX = " + max[2])

print_stock_data = stock_data[:6]
print_stock_data.append(max)

In [35]:
table_start = r"""\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{c|l|l|l|l|l|l|l}
            \toprule
            symbol & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\"""
table_end = """            \\bottomrule
        \\end{tabular}
        \\begin{tablenotes}[flushleft]
            \\footnotesize
            \\item Stylized Scores for individual stock data compared to the
            distribution of the full index data.
            Stocks in the S\&P 500 with 9216 datapoints between the
            $17^{th}$ December 1987 and the $17^{th} July 2024. The results are
            sorted by increasing average score. The Average over all the stock experiemnts
            is added above and the worst score is appended in the end.
        \\end{tablenotes}
  \\end{threeparttable}
\\end{table}"""

print(table_start)
for mu, scs, run in print_stock_data:
    ind_scores_str = " & ".join([f"${d:.2f}$" for d in scs])
    print(r"            \hline")
    print(f"            {run} & {mu:.2f} & {ind_scores_str}\\\\")

print(table_end)

\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{c|l|l|l|l|l|l|l}
            \toprule
            symbol & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\
            \hline
            AVG & 1.37 & $2.20$ & $0.57$ & $1.42$ & $1.76$ & $1.32$ & $0.98$\\
            \hline
            DIS & 0.82 & $1.13$ & $0.61$ & $0.73$ & $1.10$ & $0.84$ & $0.51$\\
            \hline
            APD & 0.84 & $1.37$ & $0.49$ & $0.67$ & $1.04$ & $0.77$ & $0.68$\\
            \hline
            CMI & 0.85 & $1.64$ & $0.55$ & $0.32$ & $1.23$ & $0.79$ & $0.55$\\
            \hline
            HON & 0.86 & $1.33$ & $0.53$ & $0.37$ & $1.02$ & $0.99$ & $0.89$\\
            \hline
            T & 0.86 & $1.54$ & $0.49$ & $0.36$ & $0.85$ & $0.84$ & $1.05$\\
            \hline
            MAX = AIG & 3.43 & $4.84$ & $0.84$ & 

In [36]:
random_split_data = zip(total_scores, scores, list(range(len(total_scores))))
random_split_data = sorted(random_split_data, key=lambda x: x[0])
avg = (np.mean(total_scores), tuple(np.mean(scores, axis=0)), "AVG")
random_split_data = list(reversed(list(reversed(random_split_data)) + [avg]))

In [37]:
table_start = r"""\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{c|l|l|l|l|l|l|l}
            \toprule
            run & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\"""
table_end = """            \\bottomrule
        \\end{tabular}
        \\begin{tablenotes}[flushleft]
            \\footnotesize
            \\item Stylized Scores for five comparisons of random splits of the 234
            stocks in the S\&P 500 with 9216 datapoints between the
            $17^{th}$ December 1987 and the $17^{th} July 2024. The results are
            sorted by increasing average score. The Average over all the five experiemnts
            is added above.
        \\end{tablenotes}
  \\end{threeparttable}
\\end{table}"""

print(table_start)
for mu, scs, run in random_split_data:
    ind_scores_str = " & ".join([f"${d:.2f}$" for d in scs])
    print(r"            \hline")
    print(f"            {run} & {mu:.2f} & {ind_scores_str}\\\\")

print(table_end)

\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{c|l|l|l|l|l|l|l}
            \toprule
            run & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\
            \hline
            AVG & 0.22 & $0.33$ & $0.18$ & $0.19$ & $0.29$ & $0.18$ & $0.14$\\
            \hline
            1 & 0.18 & $0.33$ & $0.04$ & $0.18$ & $0.24$ & $0.20$ & $0.09$\\
            \hline
            0 & 0.18 & $0.31$ & $0.03$ & $0.17$ & $0.28$ & $0.16$ & $0.13$\\
            \hline
            2 & 0.21 & $0.33$ & $0.11$ & $0.31$ & $0.25$ & $0.17$ & $0.12$\\
            \hline
            3 & 0.23 & $0.29$ & $0.26$ & $0.17$ & $0.28$ & $0.25$ & $0.15$\\
            \hline
            4 & 0.30 & $0.39$ & $0.48$ & $0.13$ & $0.43$ & $0.13$ & $0.23$\\
            \bottomrule
        \end{tabular}
        \begin{tablenotes}[flushl